In [15]:
import datetime
import random
import requests

HOME = 'http://localhost:8000/'
CARGA = 1000
# HOME = 'https://api-recintos.herokuapp.com/'

In [16]:
letras = 'ABCDEFGHIJKLMNOPQRSTUVXZ'
numeros = ''.join([str(i) for i in range(10)])
textos = 'ABCDEFGHIJKLMNOPQRSTUVXZ          abcdefghijklmnopqrstuvwxyz'
def random_str(num=3, fila=letras):
    result = ''
    for i in range(num):
        result += random.choice(fila)
    return result

   

placas = [random_str(3) + random_str(5, numeros) for i in range(100)]
reboques = [random_str(3) + random_str(5, numeros) for i in range(200)]
conteineres = [random_str(4) + random_str(7, numeros) for i in range(200)]
operadores = [random_str(11) for i in range(10)]
motoristas = [random_str(11) for i in range(50)]
textos = [random_str(random.randint(10, 20), textos) for i in range(50)]

In [17]:
pesagens = []
for r in range(CARGA):
    data = datetime.datetime.now().isoformat()
    operador = random.choice(operadores)
    conteiner = random.choice(conteineres)
    tara = random.randint(9000, 12000)
    pesobrutodeclarado = random.randint(3000, 15000)
    pesobalanca = tara + random.randint(-2000, 2000)
    placa = random.choice(placas)
    reboque = random.choice(reboques)
    texto = random.choice(textos)
    pesagem = \
    {'IDEvento': r,
     'capturaautomatica': True,
     'conteiner': random.choice(conteineres),
     'dataevento': data,
     'dataregistro': data,
     'documentotransporte': texto,
     'operadorevento': operador,
     'operadorregistro': operador,
     'pesobalanca': pesobalanca,
     'pesobrutodeclarado': pesobrutodeclarado,
     'placa': placa,
     'placasemireboque': reboque,
     'taraconjunto': tara,
     'tipodocumentotransporte': 'CE'}
    pesagens.append(pesagem)


In [18]:
pesagens[:5]

[{'IDEvento': 0,
  'capturaautomatica': True,
  'conteiner': 'MQMP0014474',
  'dataevento': '2019-05-24T17:51:41.148476',
  'dataregistro': '2019-05-24T17:51:41.148476',
  'documentotransporte': 'Kz nOQOf aRSFhH',
  'operadorevento': 'ALXRNAQNTZH',
  'operadorregistro': 'ALXRNAQNTZH',
  'pesobalanca': 13697,
  'pesobrutodeclarado': 14167,
  'placa': 'ULS33720',
  'placasemireboque': 'CPP28280',
  'taraconjunto': 11726,
  'tipodocumentotransporte': 'CE'},
 {'IDEvento': 1,
  'capturaautomatica': True,
  'conteiner': 'HIJZ3843746',
  'dataevento': '2019-05-24T17:51:41.148583',
  'dataregistro': '2019-05-24T17:51:41.148583',
  'documentotransporte': 'bsxvGmwyPT ',
  'operadorevento': 'CIMVZCIHRPE',
  'operadorregistro': 'CIMVZCIHRPE',
  'pesobalanca': 8225,
  'pesobrutodeclarado': 12191,
  'placa': 'DXC43083',
  'placasemireboque': 'KLV81845',
  'taraconjunto': 9954,
  'tipodocumentotransporte': 'CE'},
 {'IDEvento': 2,
  'capturaautomatica': True,
  'conteiner': 'TACN5862856',
  'dataevent

In [ ]:
for pesagem in pesagens:
    rv = requests.post(HOME + 'pesagemmaritimo',
                   json=pesagem)
    print(rv.status_code)
    print(rv.text)

405
"(sqlite3.IntegrityError) UNIQUE constraint failed: pesagensmaritimo.IDEvento, pesagensmaritimo.ID\n[SQL: INSERT INTO pesagensmaritimo (\"IDEvento\", dataevento, operadorevento, dataregistro, operadorregistro, \"ID\", documentotransporte, tipodocumentotransporte, conteiner, placa, placasemireboque, pesobrutodeclarado, taraconjunto, pesobalanca, capturaautomatica) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]\n[parameters: (0, '2019-05-24 17:51:41.148476', 'ALXRNAQNTZH', '2019-05-24 17:51:41.148476', 'ALXRNAQNTZH', 0, 'Kz nOQOf aRSFhH', 'CE', 'MQMP0014474', 'ULS33720', 'CPP28280', 14167, 11726, 13697, 0)]\n(Background on this error at: http://sqlalche.me/e/gkpj)"

405
"(sqlite3.IntegrityError) UNIQUE constraint failed: pesagensmaritimo.IDEvento, pesagensmaritimo.ID\n[SQL: INSERT INTO pesagensmaritimo (\"IDEvento\", dataevento, operadorevento, dataregistro, operadorregistro, \"ID\", documentotransporte, tipodocumentotransporte, conteiner, placa, placasemireboque, pesobrutodec

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento incluido"

200
"Evento

In [ ]:
for IDEvento in range(CARGA):
    rv = requests.get(HOME + 'pesagemmaritimo/' + str(IDEvento))
    print(rv.status_code)
    print(rv.json())
    
    